In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
import os
import pickle
from torchmetrics.functional.classification import *

In [2]:
df_eval = pd.DataFrame(columns=['Model','Subset Accuracy', 'Hamming Loss', 'Accuracy(exp)', 'Precision(exp)', 'Recall(exp)', 'F1-Score(exp)', 'One error', 'Coverage', 'Rangking Loss', 'Average precision', 'Accuracy(micro)', 'Accuracy (macro)', 'Precision(micro)', 'Precision(macro)', 'Recall (micro)', 'Recall (macro)', 'F1-Score (micro)', 'F1-Score (macro)', 'AUC (micro)', 'AUC (macro)'])
df_eval

,Model,Subset Accuracy,Hamming Loss,Accuracy(exp),Precision(exp),Recall(exp),F1-Score(exp),One error,Coverage,Rangking Loss,...,Accuracy(micro),Accuracy (macro),Precision(micro),Precision(macro),Recall (micro),Recall (macro),F1-Score (micro),F1-Score (macro),AUC (micro),AUC (macro)


In [3]:
model_name = ['knn', 'dt', 'ann', 'svm', 'rf', 'nb', 'dnn', 'ednn']

In [4]:
for k in model_name:
    pkl = open(f'Results/{k}_fold_pred.pkl', 'rb')
    pkl2 = open(f'Results/{k}_fold_test.pkl', 'rb')
    fold_pred = pickle.load(pkl)
    fold_test = pickle.load(pkl2)
    acc_micro, acc_macro, prec_micro, prec_macro, recall_micro, recall_macro, f1_micro, f1_macro, auc_micro, auc_macro = [], [], [], [], [], [], [], [], [], []
    type(fold_pred[0])
    for i in range(10):
        trues = torch.tensor(fold_test[i])
        if isinstance(fold_pred[i], np.ndarray):
            preds = torch.tensor(fold_pred[i])
        else:
            preds = torch.tensor(fold_pred[i].toarray())
        acc_micro.append(multilabel_accuracy(preds, trues, num_labels=7, average='micro').item())
        acc_macro.append(multilabel_accuracy(preds, trues, num_labels=7, average='macro').item())
        prec_micro.append(multilabel_precision(preds, trues, num_labels=7, average='micro').item())
        prec_macro.append(multilabel_precision(preds, trues, num_labels=7, average='macro').item())
        recall_micro.append(multilabel_recall(preds, trues, num_labels=7, average='micro').item())
        recall_macro.append(multilabel_recall(preds, trues, num_labels=7, average='macro').item())
        f1_micro.append(multilabel_f1_score(preds, trues, num_labels=7, average='micro').item())
        f1_macro.append(multilabel_f1_score(preds, trues, num_labels=7, average='macro').item())
        auc_micro.append(multilabel_auroc(preds.float(), trues.long(), num_labels=7, average='micro').item())
        auc_macro.append(multilabel_auroc(preds.float(), trues.long(), num_labels=7, average='macro').item())
    import numpy as np
    import scipy.io as sci

    def findmax(outputs):
        Max = -float("inf")
        index = 0
        for i in range(outputs.shape[0]):
            if outputs[i] > Max:
                Max = outputs[i]
                index = i
        return Max, index

    def OneError(outputs, test_target):
        test_data_num = outputs.shape[0]
        class_num = outputs.shape[1]
        num = 0
        one_error = 0
        for i in range(test_data_num):
            if sum(test_target[i]) != class_num and sum(test_target[i]) != 0:
                Max, index = findmax(outputs[i])
                num = num + 1
                if test_target[i][index] != 1:
                    one_error = one_error + 1
        return one_error / num
    def Accuracy(y_true, y_pred):
        temp = 0
        for i in range(y_true.shape[0]):
            x = sum(np.logical_and(y_true[i], y_pred[i]))
            y = sum(np.logical_or(y_true[i], y_pred[i]))
            temp +=  x/y if not np.isnan(x/y) else 0
        return temp / y_true.shape[0]

    def Recall(y_true, y_pred):
        temp = 0
        for i in range(y_true.shape[0]):
            if sum(y_true[i]) == 0:
                continue
            temp+= sum(np.logical_and(y_true[i], y_pred[i]))/ sum(y_true[i])
        return temp/ y_true.shape[0]

    def Precision(y_true, y_pred):
        temp = 0
        for i in range(y_true.shape[0]):
            if sum(y_pred[i]) == 0:
                continue
            temp+= sum(np.logical_and(y_true[i], y_pred[i]))/ sum(y_pred[i])
        return temp/ y_true.shape[0]
    subset_acc, hamming_loss, acc_exp, prec_exp, recall_exp, f1_exp, one_error, coverage, ranking_loss, avg_precision = [], [], [], [], [], [], [], [], [], []
    for i in range(10):
        trues = torch.tensor(fold_test[i])
        if isinstance(fold_pred[i], np.ndarray):
            preds = torch.tensor(fold_pred[i])
        else:
            preds = torch.tensor(fold_pred[i].toarray())
        subset_acc.append(multilabel_exact_match(preds, trues, num_labels=7).item())
        hamming_loss.append(multilabel_hamming_distance(preds, trues, num_labels=7).item())
        acc_exp.append(Accuracy(trues, preds))
        prec_exp.append(Precision(trues, preds))
        recall_exp.append(Recall(trues, preds))
        f1_exp.append(multilabel_fbeta_score(preds, trues, beta=1.0, num_labels=7).item())
        if isinstance(fold_pred[i], np.ndarray):
            one_error.append(OneError(fold_pred[i], fold_test[i]))
        else:
            one_error.append(OneError(fold_pred[i].toarray(), fold_test[i]))  
        coverage.append(multilabel_coverage_error(preds.float(), trues, num_labels=7).item())
        ranking_loss.append(multilabel_ranking_loss(preds.float(), trues, num_labels=7).item())
        avg_precision.append(multilabel_average_precision(preds.float(), trues.long(), num_labels=7).item())
    np.mean(subset_acc)
    dict_temp = {
        'Model': f'BR-{k}',
        'Subset Accuracy': np.mean(subset_acc), 
        'Hamming Loss': np.mean(hamming_loss), 
        'Accuracy(exp)': np.mean(acc_exp), 
        'Precision(exp)': np.mean(prec_exp), 
        'Recall(exp)': np.mean(recall_exp), 
        'F1-Score(exp)': np.mean(f1_exp), 
        'One error': np.mean(one_error), 
        'Coverage': np.mean(coverage), 
        'Rangking Loss': np.mean(ranking_loss), 
        'Average precision': np.mean(avg_precision), 
        'Accuracy(micro)': np.mean(acc_micro), 
        'Accuracy (macro)': np.mean(acc_macro), 
        'Precision(micro)': np.mean(prec_micro), 
        'Precision(macro)': np.mean(prec_macro), 
        'Recall (micro)': np.mean(recall_micro), 
        'Recall (macro)': np.mean(recall_macro), 
        'F1-Score (micro)': np.mean(f1_micro), 
        'F1-Score (macro)': np.mean(f1_macro), 
        'AUC (micro)': np.mean(auc_micro), 
        'AUC (macro)': np.mean(auc_macro)
    }
    dict_temp
    df_temp = pd.DataFrame([dict_temp])
    df_eval = pd.concat([df_eval, df_temp], ignore_index=True)
    df_eval
    df_eval.to_csv('Evaluation Results.csv', index=False)

C:\Users\Muhammad Rifqi Fauzi\AppData\Local\Temp\ipykernel_18772\607842060.py:117: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_eval = pd.concat([df_eval, df_temp], ignore_index=True)
